In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import os

In [3]:
X_train = pd.read_csv("../data/processed/movies_X_train_raw_imputed.csv")
X_test = pd.read_csv("../data/processed/movies_X_test_raw_imputed.csv")
y_train = pd.read_csv("../data/processed/movies_y_train.csv")
y_test = pd.read_csv("../data/processed/movies_y_test.csv")

In [62]:
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(min_df=0.01)

cvec.fit(train_data['genres'])

train_data = pd.concat([train_data.drop(columns=['genres']), pd.DataFrame(cvec.transform(train_data['genres']).todense(), columns="genre_"+ cvec.get_feature_names_out())], axis=1)
test_data = pd.concat([test_data.drop(columns=['genres']), pd.DataFrame(cvec.transform(test_data['genres']).todense(), columns="genre_"+ cvec.get_feature_names_out())], axis=1)

In [64]:
save_path = '../models/agModels-predicttop10'

predictor = TabularPredictor(label='is_top10', path=save_path, eval_metric='roc_auc').fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "../models/agModels-predicttop10/"
AutoGluon Version:  0.6.0
Python Version:     3.9.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 7 18:08:02 UTC 2019
Train Data Rows:    733
Train Data Columns: 39
Label Column: is_top10
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    63814.71 MB
	Train Data (Original)  Memory Usage: 0.67 MB (0.0% of available memory)
	Inferring data type of each featur

In [9]:
y_pred = predictor.predict(X_test)

In [10]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

AssertionError: 

In [66]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.816525,0.850464,0.248432,0.327637,49.639214,0.006410,0.000540,0.535270,2,True,14
1,CatBoost,0.813802,0.824861,0.043178,0.112680,7.140295,0.043178,0.112680,7.140295,1,True,7
2,LightGBMLarge,0.806818,0.832096,0.024298,0.025897,23.176429,0.024298,0.025897,23.176429,1,True,13
3,RandomForestEntr,0.802202,0.785158,0.145939,0.073719,0.854770,0.145939,0.073719,0.854770,1,True,6
4,LightGBM,0.801610,0.838219,0.013495,0.017337,6.392653,0.013495,0.017337,6.392653,1,True,4
5,LightGBMXT,0.790365,0.821521,0.016665,0.024217,6.600214,0.016665,0.024217,6.600214,1,True,3
6,RandomForestGini,0.789950,0.790631,0.311238,0.069135,0.830626,0.311238,0.069135,0.830626,1,True,5
7,ExtraTreesEntr,0.780303,0.780148,0.190727,0.093693,0.865788,0.190727,0.093693,0.865788,1,True,9
8,NeuralNetTorch,0.776397,0.775325,0.132900,0.137057,11.418148,0.132900,0.137057,11.418148,1,True,12
9,XGBoost,0.772727,0.830983,0.027821,0.012982,3.192827,0.027821,0.012982,3.192827,1,True,11


In [11]:
predictor.leaderboard(pd.concat([X_test, y_test], axis=1), silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.829545,0.839518,0.036544,1.113167,17.360037,0.036544,1.113167,17.360037,1,True,7
1,WeightedEnsemble_L2,0.826941,0.846753,0.722831,1.665594,67.563663,0.008739,0.000535,0.509191,2,True,14
2,LightGBMLarge,0.796520,0.817625,0.021832,0.023293,22.713506,0.021832,0.023293,22.713506,1,True,13
3,RandomForestEntr,0.795277,0.812152,8.753322,0.078739,0.873377,8.753322,0.078739,0.873377,1,True,6
4,LightGBM,0.794981,0.825417,0.013510,0.025492,5.797062,0.013510,0.025492,5.797062,1,True,4
5,RandomForestGini,0.789773,0.803989,0.113230,0.137314,4.415910,0.113230,0.137314,4.415910,1,True,5
6,LightGBMXT,0.782789,0.800371,0.016239,0.031556,5.605453,0.016239,0.031556,5.605453,1,True,3
7,XGBoost,0.770360,0.816883,0.029354,0.016744,3.725461,0.029354,0.016744,3.725461,1,True,11
8,ExtraTreesEntr,0.767164,0.764007,0.201543,0.137451,0.920551,0.201543,0.137451,0.920551,1,True,9
9,ExtraTreesGini,0.759529,0.757792,0.164347,0.113274,0.889246,0.164347,0.113274,0.889246,1,True,8


In [29]:
print(predictor.feature_metadata)

('category', [])                    :  8 | ['titleType', 'genres', 'nudity', 'violence', 'profanity', ...]
('category', ['text_as_category'])  :  1 | ['summary']
('float', [])                       :  5 | ['averageRating', 'runtimeMinutes', 'director_rank', 'writer_rank', 'actor_rank']
('int', [])                         :  5 | ['release_year', 'numVotes', 'release_date_quarter', 'release_date_month', 'release_date_week']
('int', ['binned', 'text_special']) : 15 | ['summary.char_count', 'summary.word_count', 'summary.capital_ratio', 'summary.lower_ratio', 'summary.digit_ratio', ...]
('int', ['text_ngram'])             : 68 | ['__nlp__.about', '__nlp__.after', '__nlp__.all', '__nlp__.an', '__nlp__.and', ...]


## WOw ok so lets try multimodal

In [69]:
save_path = "../models/multimodal"
predictor2 = TabularPredictor(label='is_top10', path=save_path, eval_metric='roc_auc', )
predictor2.fit(train_data.drop(columns=['numVotes', 'averageRating']), hyperparameters='multimodal')

Beginning AutoGluon training ...
AutoGluon will save models to "../models/multimodal/"
AutoGluon Version:  0.6.0
Python Version:     3.9.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 7 18:08:02 UTC 2019
Train Data Rows:    733
Train Data Columns: 37
Label Column: is_top10
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    63421.46 MB
	Train Data (Original)  Memory Usage: 0.66 MB (0.0% of available memory)
	Inferring data type of each feature based on 

Sanity Checking: 0it [00:00, ?it/s]

		CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Detailed Traceback:
Traceback (most recent call last):
  File "/home/jyl2fb/.conda/envs/netflix_proj/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 650, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/home/jyl2fb/.conda/envs/netflix_proj/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 735, in _fit_impl
    results = self._run(model, ckpt_path=self.ckpt_path)
  File "/home/jyl2fb/.conda/envs/netflix_proj/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 1166, in _run
    results = self._run_stage()
  File "/home/jyl2fb/.conda/envs/netflix_proj/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 1252, in _run_stage
    return se

In [70]:
predictor2.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.831795,0.797774,0.040268,0.079236,7.944714,0.040268,0.079236,7.944714,1,True,3
1,WeightedEnsemble_L2,0.806937,0.837291,0.173102,0.128669,18.762059,0.006702,0.000557,0.298681,2,True,8
2,XGBoost,0.794981,0.834323,0.047726,0.017510,3.268158,0.047726,0.017510,3.268158,1,True,4
3,LightGBM,0.779119,0.806679,0.035249,0.019610,6.324114,0.035249,0.019610,6.324114,1,True,1
4,LightGBMLarge,0.765980,0.810390,0.035354,0.022887,25.433338,0.035354,0.022887,25.433338,1,True,7
5,LightGBMXT,0.754616,0.808534,0.078406,0.031365,7.250506,0.078406,0.031365,7.250506,1,True,2
6,NeuralNetTorch,0.747159,0.770501,0.131167,0.139802,19.978512,0.131167,0.139802,19.978512,1,True,5
7,VowpalWabbit,0.662287,0.661781,0.147295,0.057271,0.579455,0.147295,0.057271,0.579455,1,True,6


In [14]:
predictor2.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.829545,0.839518,0.029198,0.016898,6.758718,0.029198,0.016898,6.758718,1,True,3
1,WeightedEnsemble_L2,0.826468,0.847032,1.504393,0.099913,36.082045,0.007101,0.000559,0.281726,2,True,7
2,LightGBMLarge,0.796520,0.817625,0.050982,0.017600,18.914153,0.050982,0.017600,18.914153,1,True,6
3,LightGBM,0.794981,0.825417,1.393301,0.026029,5.251225,1.393301,0.026029,5.251225,1,True,1
4,LightGBMXT,0.782789,0.800371,0.023811,0.038828,4.876223,0.023811,0.038828,4.876223,1,True,2
5,XGBoost,0.770360,0.816883,0.041711,0.012796,3.449263,0.041711,0.012796,3.449263,1,True,4
6,NeuralNetTorch,0.743371,0.768646,0.143950,0.134698,13.849812,0.143950,0.134698,13.849812,1,True,5


In [ ]:
predictor2.